In [ ]:
# ==============================================================================
# PARTE 1: ENTRENAMIENTO Y GUARDADO DEL MODELO FINALISTA
# ==============================================================================
print("--- Iniciando Parte 1: Entrenamiento del Modelo Finalista ---")

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

# --- Entrenamiento del Modelo Volumétrico Finalista ---
try:
    print("\nEntrenando el modelo volumétrico finalista...")
    df_vol = pd.read_csv('competitor_volumetric_processed.csv')
    Q1 = df_vol['price_per_liter'].quantile(0.25)
    Q3 = df_vol['price_per_liter'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    df_vol_cleaned = df_vol[df_vol['price_per_liter'] <= upper_bound].copy()
    
    features_vol = ['volume_liters', 'type_of_product', 'subindustry']
    target_vol = 'price_per_liter'
    df_vol_cleaned[features_vol] = df_vol_cleaned[features_vol].fillna('Desconocido')
    
    X = df_vol_cleaned[features_vol]
    y = df_vol_cleaned[target_vol]
    
    X_encoded = pd.get_dummies(X, columns=['type_of_product', 'subindustry'], drop_first=True)
    
    model_vol_final = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_leaf=2)
    model_vol_final.fit(X_encoded, y)
    
    joblib.dump(model_vol_final, 'final_volumetric_model2.pkl')
    joblib.dump(X_encoded.columns, 'final_volumetric_model_columns2.pkl')
    print("Modelo volumétrico guardado como 'final_volumetric_model.pkl'")

except Exception as e:
    print(f"Ocurrió un error en el entrenamiento volumétrico: {e}")

# --- Entrenamiento del Modelo por Unidades Finalista ---
try:
    print("\nEntrenando el modelo de unidades finalista...")
    df_unit = pd.read_csv('competitor_unit_processed.csv')
    Q1 = df_unit['price_per_item'].quantile(0.25)
    Q3 = df_unit['price_per_item'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    df_unit_cleaned = df_unit[df_unit['price_per_item'] <= upper_bound].copy()

    features_unit = ['total_quantity', 'type_of_product', 'subindustry']
    target_unit = 'price_per_item'
    df_unit_cleaned[features_unit] = df_unit_cleaned[features_unit].fillna('Desconocido')

    X = df_unit_cleaned[features_unit]
    y = df_unit_cleaned[target_unit]

    X_encoded = pd.get_dummies(X, columns=['type_of_product', 'subindustry'], drop_first=True)

    model_unit_final = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_leaf=2)
    model_unit_final.fit(X_encoded, y)
    
    joblib.dump(model_unit_final, 'final_unit_model2.pkl')
    joblib.dump(X_encoded.columns, 'final_unit_model_columns2.pkl')
    print("Modelo de unidades guardado como 'final_unit_model.pkl'")

except Exception as e:
    print(f"Ocurrió un error en el entrenamiento de unidades: {e}")



--- Iniciando Parte 1: Entrenamiento del Modelo Finalista ---

Entrenando el modelo volumétrico finalista...
Modelo volumétrico guardado como 'final_volumetric_model.pkl'

Entrenando el modelo de unidades finalista...
Modelo de unidades guardado como 'final_unit_model.pkl'

--- Iniciando Parte 2: Generación de Reporte de Predicciones ---

Cargando modelos desde los archivos .pkl...
¡Modelos cargados!

Se cargaron 681 productos de Wayakit para la predicción.

¡Proceso de predicción completado!
El reporte detallado se ha guardado en 'wayakit_prediction_report.csv'

--------------------------------------------------
Análisis de Rentabilidad:
Se encontraron 160 productos cuyo precio de venta predicho es MENOR o IGUAL a su costo de producción.
--------------------------------------------------

--- Vista Previa del Reporte Final ---
                                                               product_name     generic_product_type subindustry               industry  volume  cost_per_unit  

In [4]:
# ==============================================================================
# PARTE 2: CARGA DE MODELOS Y GENERACIÓN DE REPORTE CON REGLA DE NEGOCIO
# ==============================================================================
import joblib
import pandas as pd

print("\n" + "="*60)
print("--- Iniciando Parte 2: Generación de Reporte con Regla de Ganancia del 30% ---")

try:
    # Cargar los modelos y las columnas que decidimos usar (con subindustry)
    print("\nCargando modelos desde los archivos .pkl...")
    loaded_vol_model = joblib.load('final_volumetric_model2.pkl')
    loaded_unit_model = joblib.load('final_unit_model2.pkl')
    vol_model_columns = joblib.load('final_volumetric_model_columns2.pkl')
    unit_model_columns = joblib.load('final_unit_model_columns2.pkl')
    print("¡Modelos cargados!")

    # Cargar y preparar el archivo de Wayakit
    df_wayakit = pd.read_csv('wayakit_products_to_predict.csv')
    print(f"\nSe cargaron {len(df_wayakit)} productos de Wayakit para la predicción.")
    
    df_wayakit.columns = df_wayakit.columns.str.strip()
    df_wayakit['Volume_Liters'] = df_wayakit['Volume_Liters'].fillna(0)
    df_wayakit['Pack_quantity_Units'] = df_wayakit['Pack_quantity_Units'].fillna(0)
    df_wayakit.rename(columns={'SubIndustry': 'subindustry', 'Industry': 'industry', 'Generic product type': 'generic_product_type'}, inplace=True)

    report_list = []
    
    for index, row in df_wayakit.iterrows():
        is_volumetric = row['Volume_Liters'] > 0
        is_unit = row['Pack_quantity_Units'] > 0
        
        single_product_df = pd.DataFrame([row])
        
        model_predicted_price = 0
        predicted_price_per_unit = 0

        if is_volumetric:
            single_product_df.rename(columns={'Volume_Liters': 'volume_liters', 'Type_of_product': 'type_of_product'}, inplace=True)
            X_pred_encoded = pd.get_dummies(single_product_df[['volume_liters', 'type_of_product', 'subindustry']], columns=['type_of_product', 'subindustry'])
            X_pred_aligned = X_pred_encoded.reindex(columns=vol_model_columns, fill_value=0)
            predicted_price_per_unit = loaded_vol_model.predict(X_pred_aligned)[0]
            model_predicted_price = predicted_price_per_unit * row['Volume_Liters']
            
        elif is_unit:
            single_product_df.rename(columns={'Pack_quantity_Units': 'total_quantity', 'Type_of_product': 'type_of_product'}, inplace=True)
            X_pred_encoded = pd.get_dummies(single_product_df[['total_quantity', 'type_of_product', 'subindustry']], columns=['type_of_product', 'subindustry'])
            X_pred_aligned = X_pred_encoded.reindex(columns=unit_model_columns, fill_value=0)
            predicted_price_per_unit = loaded_unit_model.predict(X_pred_aligned)[0]
            model_predicted_price = predicted_price_per_unit * row['Pack_quantity_Units']
        
        # --- INICIA LA NUEVA LÓGICA DE NEGOCIO ---
        cost = row['Unit_cost_SAR']
        # 1. Calcular el precio mínimo para un 30% de ganancia
        min_profitable_price = cost * 1.30
        
        # 2. Decidir el precio final
        if model_predicted_price < min_profitable_price:
            final_price = min_profitable_price  # Aplicar el mínimo si la predicción es muy baja
        else:
            final_price = model_predicted_price # Mantener la predicción si es suficientemente alta
        
        # 3. Calcular el porcentaje de ganancia con el precio final
        profit_margin = ((final_price - cost) / cost) * 100 if cost > 0 else 0
        # --- TERMINA LA NUEVA LÓGICA DE NEGOCIO ---

        report_list.append({
            'Product_ID': row['Product_ID'],
            'product_name': row['Product_Name'],
            'generic_product_type': row.get('generic_product_type', 'N/A'),
            'subindustry': row.get('subindustry', 'N/A'),
            'industry': row.get('industry', 'N/A'),
            'volume': f"{row['Volume_Liters']} L" if is_volumetric else f"{int(row['Pack_quantity_Units'])} units",
            'cost_per_unit': cost,
            'predicted_price': round(final_price, 2), # Ahora usamos final_price
            'predicted_price_per_unit': round(predicted_price_per_unit, 2),
            'porcentaje_de_ganancia': round(profit_margin, 2)
        })

    report_df = pd.DataFrame(report_list)
    loss_products_count = (report_df['predicted_price'] <= report_df['cost_per_unit']).sum()

    report_df.to_csv('wayakit_prediction_report_final1.csv', index=False)

    print("\n¡Proceso de predicción completado con regla de ganancia!")
    print(f"El reporte detallado se ha guardado en 'wayakit_prediction_report_final1.csv'")
    print("\n" + "-"*50)
    print("Análisis de Rentabilidad:")
    print(f"Se encontraron {loss_products_count} productos cuyo precio de venta predicho es MENOR o IGUAL a su costo de producción.")
    print("-" * 50)

    print("\n--- Vista Previa del Reporte Final ---")
    print(report_df.head().to_string())

except Exception as e:
    print(f"Ocurrió un error en la predicción: {e}")


--- Iniciando Parte 2: Generación de Reporte con Regla de Ganancia del 30% ---

Cargando modelos desde los archivos .pkl...
¡Modelos cargados!

Se cargaron 681 productos de Wayakit para la predicción.

¡Proceso de predicción completado con regla de ganancia!
El reporte detallado se ha guardado en 'wayakit_prediction_report_final1.csv'

--------------------------------------------------
Análisis de Rentabilidad:
Se encontraron 0 productos cuyo precio de venta predicho es MENOR o IGUAL a su costo de producción.
--------------------------------------------------

--- Vista Previa del Reporte Final ---
     Product_ID                                                              product_name     generic_product_type subindustry               industry  volume  cost_per_unit  predicted_price  predicted_price_per_unit  porcentaje_de_ganancia
0  FP-FAI-00100                  WAYAKIT GreenTech Faith general purpose cleaner [700 mL]  General purpose cleaner       Faith  Facilities management   0